# Import

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Helper Functions

In [2]:
def create_leaf(y, ml_task):
    
    if ml_task == "regression":
        leaf = float(np.mean(y))
    else:
        counts = y.value_counts().reset_index()
        leaf = counts.iloc[0,0]
    
    return leaf


def get_potential_splits(data):
    
    X = data.drop(columns='target')
    potential_splits = {}
    columns = X.columns.tolist()
    for column in columns:

        values = X[[column]]
        unique_values = np.unique(values)
        
        potential_splits[column] = unique_values - 1
    
    return potential_splits


def calculate_gini(y):
    
    counts = y.value_counts().to_numpy()
    probabilities = counts / counts.sum()
    gini = np.sum(probabilities*(1-probabilities))
     
    return gini


def calculate_mse(y):
    
    if len(y) == 0:
        mse = 0
    else:
        mse = np.mean((y - np.mean(y)) **2)
    
    return mse


def total_impurity(data_left, data_right, metric_function):\

    n = len(data_left) + len(data_right)
    prop_left = len(data_left) / n
    prop_right = len(data_right) / n

    overall_metric =  (prop_left * metric_function(data_left['target']) 
                     + prop_right * metric_function(data_right['target']))
    
    return overall_metric


def split_data(data, column_types, split_column, split_value):
    
    type_of_feature = column_types[split_column]

    if type_of_feature == "continuous":
        data_left = data[data[split_column] <= split_value]
        data_right = data[data[split_column] >  split_value]
    
    else:
        data_left = data[data[split_column] == split_value]
        data_right = data[data[split_column] != split_value]
    
    return data_left, data_right


def determine_best_split(data, column_types, potential_splits, ml_task):

    best_overall_metric = np.inf
    for column, splits in potential_splits.items():
        for split in splits:
            
            data_left, data_right = split_data(data, column_types, split_column=column, split_value=split)
            
            if ml_task == "regression":
                node_impurity = total_impurity(data_left, data_right, metric_function=calculate_mse)
            else:
                node_impurity = total_impurity(data_left, data_right, metric_function=calculate_gini)
            
            if node_impurity <= best_overall_metric:
                best_overall_metric = node_impurity
                best_split_column = column
                best_split_value = split
    
    return best_split_column, best_split_value

# Algorithm

In [3]:
def decision_tree_algorithm(df, column_types, ml_task, min_samples=2, max_depth=5):
    
    leaves = []
    path = 'root'
    datasets = [(df,path)]
    split_conditions = []
    for current_depth in range(max_depth+1):
        next_set = []
        for dataset in datasets:
            data = dataset[0]
            path = dataset[1]
            
            if (len(data.target.unique()) == 1) or (len(data) < min_samples):
                leaf = create_leaf(data[['target']], ml_task)
                leaves.append((path,leaf))
                continue

            potential_splits = get_potential_splits(data)
            split_column, split_value = determine_best_split(data, column_types, potential_splits, ml_task)
            data_left, data_right = split_data(data, column_types, split_column, split_value)

            if len(data_left) == 0 or len(data_right) == 0:
                leaf = create_leaf(data[['target']], ml_task)
                leaves.append((path,leaf))
                continue
            print(len(data_left),len(data_right))
            split_conditions.append((path,split_column,split_value))
            next_set.append((data_left,path+',l'))
            next_set.append((data_right,path+',r'))

        datasets = next_set

    for dataset in datasets:
        data = dataset[0]
        path = dataset[1]
        leaf = create_leaf(data[['target']], ml_task)
        leaves.append((path,leaf))

    return leaves, split_conditions

# Make predictions with decision tree

def make_predictions(df, column_types, leaves, split_conditions):

    df['path'] = 'root'
    df['value'] = 0
    
    for split_condition in split_conditions:
        path = split_condition[0]
        column = split_condition[1]
        value = split_condition[2]

        if column_types[column] == "continuous":
            df.loc[(df['path']==path)&(df[column]<= value),'path'] = path+',l'
            df.loc[(df['path']==path)&(df[column]> value),'path'] = path+',r'
        else:
            df.loc[(df['path']==path)&(df[column]== value),'path'] = path+',l'
            df.loc[(df['path']==path)&(df[column]!= value),'path'] = path+',r'

    df['prediction'] = df['path'].map(dict(leaves))

    return df


def calculate_accuracy(df, column_types, ml_task, leaves, split_conditions):
    predictions = make_predictions(df, column_types, leaves, split_conditions).prediction
    
    if ml_task == 'regression':    
        predictions_array = predictions.values
        target_array = df.target.values
        metric = np.sqrt(sum((predictions_array - target_array)**2) / len(predictions_array))
        
    else:
        predictions_correct = predictions == df.target
        metric = predictions_correct.mean()
    
    return  metric

# Data Loading

In [13]:
## Read csvs
train_df = pd.read_csv('500_Person_Gender_Height_Weight_Index.csv', index_col=0)

In [14]:
train_df = train_df.fillna(0)
train_df = train_df.rename(columns={'Index':'target'})
train, val = train_test_split(train_df, test_size = 0.2)
column_types = {'Gender':'categorical','Height':'continuous','Weight':'continuous'}
ml_task = 'classifier'

# Model Training

In [26]:
leaves, split_conditions = decision_tree_algorithm(train, column_types, ml_task, min_samples=2, max_depth=5)

121 279
21 100
109 170
12 9
8 92
31 78
114 56
3 9
6 3
1 7
51 41
27 4
46 68
48 8
1 2
2 7
2 1
24 27
20 21
16 11
17 29
8 60
7 41
7 1
22 2
8 19
9 11
19 2
12 4
12 5
26 3
5 3
47 13
4 3
35 6
4 3


In [27]:
dict(leaves)

{'root,r,l,r': 5,
 'root,l,l,r,l': 4,
 'root,l,r,l,l': 1,
 'root,l,r,l,r': 0,
 'root,r,l,l,r': 4,
 'root,l,l,l,l,l': 3,
 'root,l,l,l,l,r': 2,
 'root,l,l,l,r,l': 4,
 'root,l,l,l,r,r': 3,
 'root,l,l,r,r,l': 5,
 'root,l,l,r,r,r': 4,
 'root,r,l,l,l,r': 5,
 'root,r,r,r,r,r': 5,
 'root,l,r,r,l,l,l': 2,
 'root,l,r,r,l,l,r': 2,
 'root,l,r,r,l,r,l': 0,
 'root,l,r,r,l,r,r': 1,
 'root,l,r,r,r,l,l': 3,
 'root,l,r,r,r,l,r': 2,
 'root,l,r,r,r,r,l': 2,
 'root,l,r,r,r,r,r': 3,
 'root,r,l,l,l,l,l': 5,
 'root,r,l,l,l,l,r': 4,
 'root,r,r,l,l,l,l': 4,
 'root,r,r,l,l,l,r': 4,
 'root,r,r,l,l,r,l': 3,
 'root,r,r,l,l,r,r': 2,
 'root,r,r,l,r,l,l': 4,
 'root,r,r,l,r,l,r': 5,
 'root,r,r,l,r,r,l': 4,
 'root,r,r,l,r,r,r': 4,
 'root,r,r,r,l,l,l': 5,
 'root,r,r,r,l,l,r': 4,
 'root,r,r,r,l,r,l': 5,
 'root,r,r,r,l,r,r': 5,
 'root,r,r,r,r,l,l': 4,
 'root,r,r,r,r,l,r': 4}

In [28]:
split_conditions

[('root', 'Weight', 84),
 ('root,l', 'Height', 152),
 ('root,r', 'Height', 166),
 ('root,l,l', 'Weight', 68),
 ('root,l,r', 'Weight', 50),
 ('root,r,l', 'Weight', 106),
 ('root,r,r', 'Weight', 137),
 ('root,l,l,l', 'Weight', 59),
 ('root,l,l,r', 'Weight', 78),
 ('root,l,r,l', 'Height', 181),
 ('root,l,r,r', 'Weight', 69),
 ('root,r,l,l', 'Height', 160),
 ('root,r,r,l', 'Weight', 105),
 ('root,r,r,r', 'Height', 191),
 ('root,l,l,l,l', 'Height', 147),
 ('root,l,l,l,r', 'Height', 147),
 ('root,l,l,r,r', 'Weight', 83),
 ('root,l,r,r,l', 'Height', 173),
 ('root,l,r,r,r', 'Height', 179),
 ('root,r,l,l,l', 'Weight', 95),
 ('root,r,r,l,l', 'Height', 177),
 ('root,r,r,l,r', 'Height', 171),
 ('root,r,r,r,l', 'Weight', 139),
 ('root,r,r,r,r', 'Weight', 158),
 ('root,l,r,r,l,l', 'Weight', 67),
 ('root,l,r,r,l,r', 'Weight', 55),
 ('root,l,r,r,r,l', 'Weight', 77),
 ('root,l,r,r,r,r', 'Weight', 83),
 ('root,r,l,l,l,l', 'Height', 151),
 ('root,r,r,l,l,l', 'Height', 173),
 ('root,r,r,l,l,r', 'Height', 

# Prediction

In [29]:
predict = make_predictions(val, column_types, leaves, split_conditions)
predict

,Height,Weight,target,path,value,prediction
Gender,,,,,,
Female,153,51,2,"root,l,r,r,l,l,l",0,2
Female,153,149,5,"root,r,l,r",0,5
Male,141,80,5,"root,l,l,r,r,l",0,5
Female,156,106,5,"root,r,l,l,l,r",0,5
Male,185,60,1,"root,l,r,r,l,r,r",0,1
...,...,...,...,...,...,...
Male,144,80,4,"root,l,l,r,r,l",0,5
Female,189,124,4,"root,r,r,l,r,r,l",0,4
Male,195,98,3,"root,r,r,l,l,r,l",0,3


In [30]:
accuracy = calculate_accuracy(val, column_types, ml_task, leaves, split_conditions)
accuracy

0.76